In [ ]:
import numpy as np
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten, BatchNormalization
from keras import optimizers
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications.inception_v3 import InceptionV3
from PIL import ImageFile

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

IMG_SIZE = 300
LEARNING_RATE = 0.001
EPOCHS = 20
BATCH_SIZE = 32

TRAIN_SIZE = 6710
TEST_SIZE = 843

TRAIN_DIR = 'C:/Users/SAMSUNG/Documents/fresh_meat/dataset/train'
TEST_DIR = 'C:/Users/SAMSUNG/Documents/fresh_meat/dataset/test'

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE)

test_gen = datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE)

In [ ]:
# Initialize Model
inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [ ]:
# Only allow training on top layers
for layer in inception_model.layers:
    layer.trainable = False

x = inception_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(5, activation='softmax', name='predictions')(x)

# Combine base model with top layers
model = Model(inception_model.input, predictions)

model.summary()

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [ ]:
training = model.fit_generator(
    generator=train_gen,
    steps_per_epoch= TRAIN_SIZE // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=test_gen,
    validation_steps= TEST_SIZE // BATCH_SIZE)

In [ ]:
model.save('models/inception-meat.h5')